# Enwex

https://enwex.com/wp-content/uploads/2024/03/ENWEX_wind.pdf


In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

sk.set_file_destination("enwex_example", force=False)
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Get the Source Meteorology

### Set Shared Parameters


In [ ]:
start_date = "2024-03-01"

if True:
    # Generate a fast and approximate timeseries
    members = 11
    day_count = 35  # Matches the NOAA_GEFS lead length
else:
    # Generate a more comprehensive timeseries
    members = 101
    day_count = 366  # Salient can forecast up to a year in advance

# Not recommended to change these parameters:
force = False
vars = ["temp", "tsi", "wspd100"]
freq = "hourly"
end_date = np.datetime64(start_date) + np.timedelta64(day_count, "D")

### Set geographic bounds

Enwex trades temperature, wind, and solar defivatives based off 12 grid points throughout Germany.

In [ ]:
# fmt: off
loc = sk.Location(location_file = sk.upload_location_file(
    lats =  [48.5, 49.0,  52.5, 50.5, 53.75,   52.5, 51.5,  50.00, 51.0, 52.00,  54.25, 51.0],
    lons =  [ 9.0, 11.5,  13.5,  9.0, 12.50,    9.0,  7.5,   7.25, 13.5, 11.75,   9.75, 11.0],
    names = ["BW","BAY","BB_B", "HE",  "MV","NS_HB", "NW","RP_SL",  "S",  "SA","SH_HH", "TH"],
    geoname = "enwex",
    force = False,
    weight_wnd = [ 3.1, 4.6,14.0,4.1,6.4,21.1,11.4,7.7,2.3,9.5,12.7,3.1],
    weight_sun = [12.6,27.1, 8.3,4.5,5.1, 8.6,11.1,5.7,4.3,5.8, 3.5,3.4],
    weight_tmp = [13.4,15.9, 7.5,7.6,1.9,10.5,21.5,6.1,4.8,2.6, 5.7,2.5],
    long_name = ["Baden-Wuerttemberg", "Bayern", "Brandenburg & Berlin", "Hessen",
        "Mecklenburg-Vorpommern", "Niedersachsen & Bremen", "Nordrhein-Westfalen",
        "Rheinland-Pfalz", "Sachsen", "Sachsen-Anhalt", "Schleswig-Holstein & Hamburg",
        "Thueringen"]),
    )
# fmt: on
# loc.plot_locations(title="ENWEX Weather Index Locations", weight="weight_wnd")
print(loc)

location file: enwex.csv


### Get Forecast

This is what the forecast predicted as of the forecast date

In [ ]:
fcst_met = sk.merge_location_data(
    xr.open_dataset(
        sk.downscale(
            loc=loc,
            variables=vars,
            date=start_date,
            length=day_count,
            frequency=freq,
            members=members,
            force=force,
        )
    ),
    loc_file=loc,
)

print(fcst_met)

<xarray.Dataset> Size: 1MB
Dimensions:        (location: 12, time: 839, ensemble: 11)
Coordinates:
  * location       (location) object 96B 'BW' 'BAY' 'BB_B' ... 'SA' 'SH_HH' 'TH'
  * time           (time) datetime64[ns] 7kB 2024-03-01T01:00:00 ... 2024-04-...
  * ensemble       (ensemble) int64 88B 0 1 2 3 4 5 6 7 8 9 10
    forecast_date  datetime64[ns] 8B ...
    lat            (location) float64 96B ...
    lon            (location) float64 96B ...
    analog         (ensemble, time) datetime64[ns] 74kB ...
Data variables:
    weight_wnd     (location) float64 96B 3.1 4.6 14.0 4.1 ... 2.3 9.5 12.7 3.1
    weight_sun     (location) float64 96B 12.6 27.1 8.3 4.5 ... 4.3 5.8 3.5 3.4
    weight_tmp     (location) float64 96B 13.4 15.9 7.5 7.6 ... 4.8 2.6 5.7 2.5
    long_name      (location) object 96B 'Baden-Wuerttemberg' ... 'Thueringen'
    wspd100        (ensemble, time, location) float32 443kB ...
    temp           (ensemble, time, location) float32 443kB ...
    tsi            (

### Get Historical "Truth"

This is what actually happened, according to ECMWF.

In [ ]:
hist_met = sk.merge_location_data(
    sk.load_multihistory(
        sk.data_timeseries(
            loc=loc,
            variable=vars,
            field="vals",
            start=start_date,
            end=end_date,
            frequency=freq,
            force=force,
        )
    ),
    loc_file=loc,
)
print(hist_met)

<xarray.Dataset> Size: 250kB
Dimensions:     (location: 12, time: 841)
Coordinates:
  * location    (location) object 96B 'BW' 'BAY' 'BB_B' ... 'SA' 'SH_HH' 'TH'
  * time        (time) datetime64[ns] 7kB 2024-03-01 ... 2024-04-05
    lat         (location) float64 96B 48.5 49.0 52.5 50.5 ... 52.0 54.25 51.0
    lon         (location) float64 96B 9.0 11.5 13.5 9.0 ... 11.75 9.75 11.0
Data variables:
    weight_wnd  (location) float64 96B 3.1 4.6 14.0 4.1 6.4 ... 2.3 9.5 12.7 3.1
    weight_sun  (location) float64 96B 12.6 27.1 8.3 4.5 5.1 ... 4.3 5.8 3.5 3.4
    weight_tmp  (location) float64 96B 13.4 15.9 7.5 7.6 1.9 ... 4.8 2.6 5.7 2.5
    long_name   (location) object 96B 'Baden-Wuerttemberg' ... 'Thueringen'
    temp        (time, location) float64 81kB 1.969 1.581 4.835 ... 7.983 9.654
    tsi         (time, location) float64 81kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    wspd100     (time, location) float64 81kB 3.072 0.7619 6.085 ... 8.561 8.257


## Calculate Enwex Index

In [ ]:
def calc_enwex(met: xr.Dataset) -> xr.Dataset:
    """Calculate the ENWEX indices for a given forecast dataset.

    Args:
        met: A Dataset containing the forecast meteorological variables and weights:
            - wspd100: Wind speed at 100m height [m/s]
            - tsi: Total solar irradiance [W/m^2]
            - temp: Temperature [degC]
            - weight_wnd: Wind weight for each location
            - weight_sun: Solar weight for each location
            - weight_tmp: Temperature weight for each location
            - location: vector of geo points, same length as `weight` vectors

    Returns:
        A Dataset containing the ENWEX indices:
            - enwex_wind: ENWEX wind index [%]
            - enwex_solar: ENWEX solar index [%]
            - enwex_temperature: ENWEX temperature index [degC]
    """
    src_wind = met["wspd100"].weighted(met["weight_wnd"]).mean(dim="location")
    src_wind.attrs = {"long_name": "Weighted Wind Speed", "units": "m/s"}

    return xr.Dataset(
        {
            "src_wind": src_wind,
            "enwex_wind": calc_enwex_wind(met["wspd100"], met["weight_wnd"]),
            "enwex_solar": calc_enwex_solar(met["tsi"], met["weight_sun"]),
            "enwex_temperature": calc_enwex_temp(met["temp"], met["weight_tmp"]),
        }
    )


def calc_enwex_wind(wind: xr.DataArray, weight: xr.DataArray) -> xr.DataArray:
    """Calculate the ENWEX wind index for a given forecast dataset."""
    idx_loc = 100 * (
        (0.92 + 0.05) / (1.0 + np.exp(3.2 - 0.529 * (wind - 2.5) - 0.0074)) - 0.05
    ).clip(min=0)
    idx = idx_loc.weighted(weight).mean(dim="location")
    idx.attrs["units"] = "%"
    idx.attrs["long_name"] = "ENWEX Wind Index"
    return idx


def calc_enwex_solar(tsi: xr.DataArray, weight: xr.DataArray) -> xr.DataArray:
    """Calculate the ENWEX solar index for a given forecast dataset."""
    idx_loc = 100 * (0.71 * (tsi / 1000))
    idx = idx_loc.weighted(weight).mean(dim="location")
    idx.attrs["units"] = "%"
    idx.attrs["long_name"] = "ENWEX Solar Index"
    return idx


def calc_enwex_temp(temp: xr.DataArray, weight: xr.DataArray) -> xr.DataArray:
    """Calculate the ENWEX temperature index for a given forecast dataset."""
    idx = temp.weighted(weight).mean(dim="location")
    idx.attrs["units"] = "degC"
    idx.attrs["long_name"] = "ENWEX Temperature Index"
    return idx

In [ ]:
fcst = calc_enwex(fcst_met)
print(fcst)

<xarray.Dataset> Size: 376kB
Dimensions:            (time: 839, ensemble: 11)
Coordinates:
  * time               (time) datetime64[ns] 7kB 2024-03-01T01:00:00 ... 2024...
  * ensemble           (ensemble) int64 88B 0 1 2 3 4 5 6 7 8 9 10
    forecast_date      datetime64[ns] 8B 2024-03-01
    analog             (ensemble, time) datetime64[ns] 74kB NaT ... 2018-03-2...
Data variables:
    src_wind           (ensemble, time) float64 74kB nan nan ... 4.46 4.496
    enwex_wind         (ensemble, time) float64 74kB nan nan ... 9.317 7.661
    enwex_solar        (ensemble, time) float64 74kB 0.0 0.0 ... 0.0 0.0
    enwex_temperature  (ensemble, time) float64 74kB nan nan ... 2.326 1.892


In [ ]:
hist = calc_enwex(hist_met)
print(hist)

<xarray.Dataset> Size: 34kB
Dimensions:            (time: 841)
Coordinates:
  * time               (time) datetime64[ns] 7kB 2024-03-01 ... 2024-04-05
Data variables:
    src_wind           (time) float64 7kB 6.72 6.855 6.831 ... 8.244 8.204 8.098
    enwex_wind         (time) float64 7kB 25.45 26.12 25.45 ... 40.04 39.6 38.34
    enwex_solar        (time) float64 7kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    enwex_temperature  (time) float64 7kB 4.17 3.929 3.65 ... 10.2 10.07 10.01


In [ ]:
hist.plot.scatter(x="src_wind", y="enwex_wind", alpha=0.1)
plt.show();

In [ ]:
from matplotlib.lines import Line2D

# Plot lead week 3:
plot_time = slice(14 * 24, 21 * 24)

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 3))

for idx, ax in zip(["enwex_wind", "enwex_solar"], axes):
    fcst_h = (
        fcst[idx]
        .isel(time=plot_time)
        .plot.line(x="time", ax=ax, color=(0.5, 0.5, 0.5, 0.2), add_legend=False)
    )
    hist_h = (
        hist[idx]
        .isel(time=plot_time)
        .plot.line(x="time", ax=ax, color="dodgerblue", add_legend=False)
    )

    if ax == axes[0]:
        ax.legend(
            handles=[
                Line2D([0], [0], color=(0.5, 0.5, 0.5, 0.2), lw=2, label="Forecast ensemble"),
                Line2D([0], [0], color="dodgerblue", lw=2, label="Settlement"),
            ]
        )

plt.show()

## Validate Forecast Accuracy


In [ ]:
mae = abs(hist - fcst).mean(dim="ensemble")
mae = mae.assign_coords(lead=("time", (mae["time"] - mae["forecast_date"]).values))

print(mae)

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 15))

# Plot each variable on a separate axis
mae["enwex_wind"].plot(ax=axes[0], x="lead", color="black")
axes[0].set_title("ENWEX Wind MAE")

mae["enwex_solar"].plot(ax=axes[1], x="lead", color="black")
axes[1].set_title("ENWEX Solar MAE")

mae["enwex_temperature"].plot(ax=axes[2], x="lead", color="black")
axes[2].set_title("ENWEX Temperature MAE")

# Adjust layout
plt.tight_layout()
plt.show()

<xarray.Dataset> Size: 40kB
Dimensions:            (time: 839)
Coordinates:
  * time               (time) datetime64[ns] 7kB 2024-03-01T01:00:00 ... 2024...
    forecast_date      datetime64[ns] 8B 2024-03-01
    lead               (time) timedelta64[ns] 7kB 01:00:00 ... 34 days 23:00:00
Data variables:
    src_wind           (time) float64 7kB nan nan 0.3167 ... 2.049 1.79 1.841
    enwex_wind         (time) float64 7kB nan nan 3.077 ... 19.38 16.59 17.56
    enwex_solar        (time) float64 7kB 4.218e-07 0.0 ... 4.482e-07 4.482e-07
    enwex_temperature  (time) float64 7kB nan nan 0.3078 ... 5.221 5.619 5.693


## Appendix: Validate Calculation Methodology


In [ ]:
wind_file = os.path.join(sk.get_file_destination(), "ENWEXwind_since2022.xlsx")
if os.path.exists(wind_file):
    if "index_ref" not in locals():
        # read_excel is heavyweight, only read this if needed
        assert os.path.exists(wind_file)
        index_ref = (
            pd.read_excel(wind_file, sheet_name="ENWEXwind_2022", skiprows=17, usecols="C:E")
            .set_index("DateTimeUTC")
            .merge(hist["enwex_wind"].to_dataframe(), left_index=True, right_index=True)
        )

    plot_data = index_ref.iloc[:100, [1, 2]]  # Adjust column indices if needed

    plt.plot(plot_data.index, plot_data.iloc[:, 0], label="Reference")
    plt.plot(plot_data.index, plot_data.iloc[:, 1], label="As-Calculated")
    plt.legend()

    print(index_ref)

In [ ]:
wind_file = os.path.join(sk.get_file_destination(), "ENWEXwind_since2022.xlsx")
if os.path.exists(wind_file):
    if "wind_ref" not in locals():
        wind_ref = pd.read_excel(wind_file, sheet_name="Windspeed_100m_Ecmwf", usecols="B:N")
        wind_ref = wind_ref.set_index("DateTimeUTC")
        wind_ref = wind_ref.rename(
            columns={
                "5425_0975_1270": "SH_HH",
                "5375_1250_0640": "MV",
                "5250_0900_2110": "NS_HB",
                "5250_1350_1400": "BB_B",
                "5200_1175_0950": "SA",
                "5150_0750_1140": "NW",
                "5100_1350_0230": "S",
                "5100_1100_0310": "TH",
                "5050_0900_0410": "HE",
                "5000_0725_0770": "RP_SL",
                "4900_1150_0460": "BAY",
                "4850_0900_0310": "BW",
            }
        )
    wind_era = (
        hist_met.wspd100.drop_vars(["lat", "lon"])
        .to_dataframe()
        .reset_index()
        .pivot(index="time", columns="location", values="wspd100")
    )

    wind_mrg = pd.merge(
        wind_ref["MV"],
        wind_era["MV"],
        left_index=True,
        right_index=True,
        how="inner",
        suffixes=("_opr", "_era"),
    )

    if False:
        plt.plot(wind_ref.iloc[0:100]["MV"], label="ECMWF Operational")
        plt.plot(wind_xxx.iloc[0:100]["MV"], label="ERA5")
        plt.title("Mecklenburg-Vorpommern")
        plt.ylabel("Wind Speed @ 100m [m/s]")
        plt.legend()
    else:
        correlation = wind_mrg["MV_opr"].corr(wind_mrg["MV_era"])
        # create a scatterplot of wind_ref vs wind_xxx at location MV
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(wind_mrg["MV_opr"], wind_mrg["MV_era"], alpha=0.1)
        ax.plot([0, 20], [0, 20], "k--")
        ax.set_aspect("equal", adjustable="box")
        plt.xlabel("ECMWF Operational")
        plt.ylabel("ERA5")
        plt.title("Wind @ 100m [m/s]")
        ax.text(0, 20, f"Correlation: {correlation:.2f}", fontsize=12)
        # plt.legend()
        # plt.grid(True)
        plt.show()

## Appendix: Export Forecasts

Save as a CSV for external consumption.

In [ ]:
out_path = os.path.join(sk.get_file_destination(), f"enwex-de-{start_date}.csv")
out_drop = ["long_name", "analog", "weight_wnd", "weight_sun", "weight_tmp"]
out = fcst_met.drop_vars(out_drop, errors="ignore")

for var in ["temp", "tsi", "wspd100"]:
    # Replace any missing values with historical actuals
    out[var] = out[var].fillna(hist_met[var]).round(4)

# To assist in adding to a database, distinguish each date's forecast
out = out.expand_dims("forecast_date")

if True:
    # Write a fast data sample single-day subset of the forecast
    out_time = len(out.time) // 2
    out = out.isel(time=slice(out_time - 12, out_time + 12))

out = out.to_dataframe(dim_order=["forecast_date", "time", "location", "ensemble"])
out.to_csv(out_path)

print(f"Wrote met forecast to {out_path}")
print(out)

Wrote met forecast to enwex_example/enwex-de-2024-03-01.csv
                                                      lat   lon  wspd100     temp  tsi
forecast_date time                location ensemble                                   
2024-03-01    2024-03-18 00:00:00 BW       0         48.5   9.0   2.4132  -2.7049  0.0
                                           1         48.5   9.0   3.7900   5.8971  0.0
                                           2         48.5   9.0   7.5274   5.3299  0.0
                                           3         48.5   9.0   3.8520   5.4125  0.0
                                           4         48.5   9.0   4.6582   2.6851  0.0
...                                                   ...   ...      ...      ...  ...
              2024-03-18 23:00:00 TH       6         51.0  11.0  10.2662   8.4212  0.0
                                           7         51.0  11.0   9.3928  11.8993  0.0
                                           8         51.0  11.0   8.30